## Cover Song Identification using comparison of hierarchical structure for covers80

### Library Importing

### Dill session

In [ ]:
dill.dump_session('sets.db')

In [ ]:
dill.load_session('sets.db')

### Load audio

In [ ]:
#Choose directory containing audiofiles
directory = '../../../Datasets/covers80'

In [ ]:
#Read all paths in specified directory
all_dirs = []
all_names= []
for root, dirs, files in os.walk(directory):
        for name in files:
            if (('.wav' in name) or ('.aif' in name) or ('.mp3' in name)):
                filepath = os.path.join(root, name)
                all_dirs.append(filepath)
                all_names.append(name[:-4])
file_no = len(all_names)
#Load all audiofiles and store in array
all_audio = []
for f in range(file_no):
    y, sr = librosa.load(all_dirs[f], sr=22050, mono=True)
    all_audio.append((y, sr))
    sys.stdout.write("\rLoaded %i/%s pieces." % ((f+1), str(file_no)))
    sys.stdout.flush()

### Hierarchical structure decomposition

In [ ]:
#CQT config
BINS_PER_OCTAVE = 12 * 3
N_OCTAVES = 7

#Approximations
kmin = 2
kmax = 8

all_struct = []

for i in range(file_no):
    y, sr = all_audio[i]
   
    C = librosa.amplitude_to_db(np.abs(librosa.cqt(y=y, sr=sr,
                                        bins_per_octave=BINS_PER_OCTAVE,
                                        n_bins=N_OCTAVES * BINS_PER_OCTAVE)),
                            ref=np.max)
    
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
    all_beat_times.append(librosa.frames_to_time(librosa.util.fix_frames(beats, x_min=0, x_max=C.shape[1]), sr=sr))

    Csync = librosa.util.sync(C, beats, aggregate=np.median)

    Cstack = librosa.feature.stack_memory(Csync, 4)
    
    R = librosa.segment.recurrence_matrix(Cstack, width=3, mode='affinity', sym=True)

    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    Rf = librosa.segment.path_enhance(Rf, 15)
    
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    Msync = librosa.util.sync(mfcc, beats)

    path_distance = np.sum(np.diff(Msync, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)

    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)

    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)

    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)

    A = mu * Rf + (1 - mu) * R_path
    
    L = scipy.sparse.csgraph.laplacian(A, normed=True)
    
    #eigendecomposition
    evals, evecs = scipy.linalg.eigh(L)
    #eigenvector filtering
    evecs = scipy.ndimage.median_filter(evecs, size=(9, 1))
    #normalization
    Cnorm = np.cumsum(evecs**2, axis=1)**0.5
    dist_set = []
    for k in range(kmin, kmax):
        Xs = evecs[:, :k] / Cnorm[:, k-1:k]
        distance = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(Xs, metric='euclidean'))
        dist_set.append(distance)
    all_struct.append(dist_set)    

    sys.stdout.write("\rComputed for %i/%s pieces." % ((f+1), str(file_no)))
    sys.stdout.flush()

### Flatten upper triangle of each approximation